# Spark SQL: consultas

#### Máster en Data Science y Big Data, AFI Escuela de Finanzas
#### Claudia Quintana Wong

## Descripción de ficheros

Fichero albums.tsv:
* id: Identificador único del disco.
* title: Título del disco.

Fichero artists.tsv:

* id: Identificador único del artista.
* name: Nombre del artista.
* hotness: Nivel de popularidad del artista.
* familiarity: Reconocimiento del artista.
* location: Ubicación del artista.


Fichero songs.tsv:

* id: Identificador único de la canción.
* title: Título de la canción.
* year: Año de publicación de la canción.
* hotness: Popularidad de la canción.
* id_artist: Identificador único del artista de la canción.
* id_album: Identificador único del álbum de la canción.
* duration: Duración en segundos de la canción.
* end_of_fade_in: Segundo de la canción en el que termina el fade in.
* start_of_fade_out: Segundo de la canción en el que empieza el fade out.
* tempo: Tempo de la canción.
* time_signature: Número de tiempos por compás de la canción.
* key: Escala de la canción (de 0 a 11).
* loudness: Volumen de la canción.
* mode: Tipo de escala de la canción (mayor = 0 o menor = 1)
* style: Estilo de la canción.

## Carga de datos

In [1]:
from pyspark.sql.types import Row
from pyspark.sql import functions as F

In [2]:
def albumParser(line):
    fields = line.split('\t')
    return Row(id = fields[0], title = fields[1])

def artistParser(line):
    fields = line.split('\t')
    return Row(id=fields[0], 
               name = fields[1], 
               hotness = float(fields[2]), 
               familiarity = fields[3], 
               location = fields[4])

def songParser(line):
    fields = line.split('\t')
    return Row(id = fields[0],
               title = fields[1],
               year = int(fields[2]),
               hotness = float(fields[3]) if fields[3] != 'NA' else 0,
               id_artist = fields[4],
               id_album = fields[5],
               duration = float(fields[6]),
               end_of_fade_in = float(fields[7]),
               start_of_fade_out = float(fields[8]),
               tempo = float(fields[9]),
               time_signature = fields[10],
               key = fields[11],
               loudness = float(fields[12]),
               mode = int(fields[13]),
               style = fields[14])

In [3]:
data_folder = './data'

albums = sc.textFile(f'{data_folder}/albums.tsv')
artists = sc.textFile(f'{data_folder}/artists.tsv')
songs = sc.textFile(f'{data_folder}/songs.tsv')

In [4]:
albums_rows = albums.map(albumParser)
artists_rows = artists.map(artistParser)
songs_rows = songs.map(songParser)

In [5]:
album_df = spark.createDataFrame(albums_rows)
artist_df = spark.createDataFrame(artists_rows)
song_df = spark.createDataFrame(songs_rows)

In [6]:
album_df.createOrReplaceTempView('Albums')
artist_df.createOrReplaceTempView('Artist')
song_df.createOrReplaceTempView('Song')

### 1. ¿Cuál es el estilo más rápido (tempo) en media?

Para dar respuesta a esta consulta seguiremos los siguientes pasos:

1. Agrupar las canciones por estilo
2. Hallar la media en cada grupo teniendo en cuenta la variable *tempo*
3. Ordenar de manera ascendente de acuerdo a la media calculada.
4. Devolver solo el primer elemento de la lista

In [7]:
spark.sql("""SELECT style, AVG(tempo) AS avg_tempo  
          FROM Song 
          GROUP BY style 
          ORDER BY avg_tempo 
          LIMIT 1""").show()

+--------+---------+
|   style|avg_tempo|
+--------+---------+
|rebetika|   47.447|
+--------+---------+



### 2. ¿Cuales son los 5 artistas, ubicados en UK (cualquier territorio de UK), con mayor número de canciones en escala menor (mode = 1)?

Para dar respuesta a esta interrogante seguiremo los siguientes pasos:

1. Obtener los artistas ubicados en UK (cualquier territorio de UK)
2. Seleccionar las canciones en escala menor
3. Mezclar los artistas de UK con las canciones seleccionadas
4. Hacer un conteo de canciones por artista
5. Ordenar descendientemente los artistas de acuerdo al conteo y seleccionar los 5 primeros


In [8]:
spark.sql("""SELECT A.id, A.name, COUNT(A.id)
        FROM Artist A INNER JOIN Song S ON A.id=S.id_artist
        WHERE (A.location like '%UK%' OR A.location like '%United Kingdom%' OR A.location like '%England%') AND S.mode = 1
        GROUP BY A.id, A.name
        ORDER BY COUNT(A.id) DESC
        LIMIT 5
        """
        ).show()

+------------------+------------------+---------+
|                id|              name|count(id)|
+------------------+------------------+---------+
|AR9W3X91187FB3994C|      Phil Collins|        9|
|ARH6W4X1187B99274F|         Radiohead|        7|
|ARFCUN31187B9AD578|The Rolling Stones|        7|
|ARAIABB1187B9AC6E2|              Seal|        6|
|ARD8JVH1187FB4DA04|       Bad Company|        6|
+------------------+------------------+---------+



### 3. Desde 1970 hasta hoy, ¿las canciones son más rápidas (tempo), altas (loudness) y cortas (duration) en media? Ordena los resultados por año ascendente.

Se seguirán los siguientes pasos:

1. Seleccionar las canciones de 1970 hasta hoy
2. Seleccionar las columnas: *tempo*, *loudness*, *duration*
3. Agrupar las canciones por año y calcular la media de las variables selecciondas
4. Ordenar los grupos según el año de lanzamiento

In [9]:
spark.sql("""SELECT S.year, 
                    AVG(S.tempo) AS avg_tempo, 
                    AVG(S.loudness) AS avg_loudness, 
                    AVG(S.duration) AS avg_duration
            FROM Song S            
            WHERE S.year >= 1970
            GROUP BY S.year
            ORDER BY S.year ASC""").show()

+----+------------------+-------------------+------------------+
|year|         avg_tempo|       avg_loudness|      avg_duration|
+----+------------------+-------------------+------------------+
|1970|121.34628571428571| -11.92847619047619|231.42578619047623|
|1971|         136.16196|-12.153000000000002|259.55428919999997|
|1972|129.17204166666667|-11.719291666666665|238.54539750000004|
|1973|        116.356125|-11.711541666666667|294.16444416666667|
|1974|125.08609090909088|-10.670681818181817|239.49134636363635|
|1975|125.41183333333333|-11.249541666666666| 277.4406354166667|
|1976|137.26139999999998|           -11.6584| 210.9940493333333|
|1977|139.33685714285713|-11.820114285714284|255.30692800000003|
|1978|         134.38385|           -10.1125|247.85456749999997|
|1979|137.51694444444445|-11.879083333333334| 226.0566886111111|
|1980|126.89337499999999|       -11.09853125|     210.438730625|
|1981|127.96074999999999|-11.570444444444446|211.69224499999999|
|1982|         125.14522|

### 4. ¿Cuál es el estilo que más abusa de los efectos de fade in y fade out (mayor número de segundos desde inicio al final del fade in más desde el inicio del fade out al final de la canción)?

Se utiliza la misma expresión de cálculo empleada para resolver la consulta en Spark Core.

$$end\_of\_fade\_in + duration - start\_of\_fade\_out$$

Para encontrar el estilo que más abusa de estos efectos se determina la canción que cumple este requisito y se devuelve el estilo asociado.

Los pasos a seguir son:

1. Calcular la expresión para cada canción de la base de datos
2. Ordenar de manera descendente según la métrica calculada
3. Devolver el estilo y el título de la primera observación en el resultado anterior

In [10]:
spark.sql("""SELECT S.style, S.title, (S.end_of_fade_in + S.duration - S.start_of_fade_out) as fades_effect
            FROM Song S
            ORDER BY fades_effect DESC
            LIMIT 1
            """).show()

+----------------+------+------------+
|           style| title|fades_effect|
+----------------+------+------------+
|industrial dance|bereit|   123.34402|
+----------------+------+------------+



### Intepretación 2
El estilo que más abusa de los efectos fades es aquel en que sus canciones abusan más de este efecto en media

In [11]:
spark.sql("""SELECT S.style, AVG(S.end_of_fade_in + S.duration - S.start_of_fade_out) as avg_fades_effect
FROM Song S
GROUP BY S.style
ORDER BY avg_fades_effect DESC
LIMIT 1

""").show()

+----------------+-----------------+
|           style| avg_fades_effect|
+----------------+-----------------+
|industrial dance|61.67226499999998|
+----------------+-----------------+



### 5. ¿Cual es la canción más popular (hotness) de los 5 artistas más populares (hotness)?

Para dar respuesta a esta consulta se emplea una subconsulta. La subconsulta se encarga de calcular el valor máximo de la variable *hotness* entre las canciones interpretadas por determinado artista que recibe de la consulta general. De manera tal, que esta subconsulta se ejecuta por cada uno de los artistas. Posteriormente se hace el match basado en este valor máximo encontrado.

In [12]:
spark.sql("""

SELECT A.id, A.name, A.hotness AS artist_hotness, S.id, S.title, S.hotness AS song_hotness
FROM Artist A INNER JOIN Song S ON A.id=S.id_artist
WHERE S.hotness = 
    (
        SELECT MAX(S1.hotness)
        FROM Artist A1 INNER JOIN Song S1 ON A1.id=S1.id_artist
        WHERE A1.id = A.id
    )
ORDER BY artist_hotness DESC
LIMIT 5

""").show()

+------------------+---------------+--------------+------------------+--------------------+------------+
|                id|           name|artist_hotness|                id|               title|song_hotness|
+------------------+---------------+--------------+------------------+--------------------+------------+
|ARRH63Y1187FB47783|     Kanye West|   1.082502557|SOJMUAN12AB0183911|       Street Lights| 0.814517241|
|ARF8HTQ1187B9AE693|      Daft Punk|   1.021255588|SONJBQX12A6D4F8382|             Da Funk|   0.8622545|
|ARTDQRC1187FB4EFD4|Black Eyed Peas|   1.005941966|SOCHRXB12A8AE48069|Let's Get It Started| 0.624425493|
|ARS54I31187FB46721|   Taylor Swift|   0.922412443|SOTNWCI12AAF3B2028| The Way I Loved You| 0.853828893|
|ARJ7KF01187B98D717|       Coldplay|   0.916053228|SOEHTZE12A6310F0F2|          One I Love| 0.810263613|
+------------------+---------------+--------------+------------------+--------------------+------------+

